# Import libraries

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/ROBT613_BMI_Final_Project

/content/drive/MyDrive/ROBT613_BMI_Final_Project


In [3]:
!pip install mne==0.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mne: filename=mne-0.18.0-py3-none-any.whl size=6313874 sha256=a8e93268ad9d50d60d3f6ac4cf4ef907114cc092c3a792bcde2d7b4cd249583c
  Stored in directory: /root/.cache/pip/wheels/50/55/a4/d95e3c6ab9370b77f9c871e9882a9578a2195c64ede88694ff
Successfully built mne


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import itertools
import pandas as pd
import pickle
import numpy as np
import mne
from sklearn.preprocessing import StandardScaler

from nu_smrutils import loaddat,load_pooled,augment_dataset,crop_data,train_model

%matplotlib inline
%load_ext autoreload
%autoreload 2
torch.manual_seed(0)

# Select device

In [5]:
#by default
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())

Your GPU device name : Tesla T4


# Select Dataset

## 1. BNCI2014004

In [6]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


# Load subject specific data, preprocess data (filtering, data augmentation, data cropping, standard scaling), and use Pytorch dataloaders

Function for standard scaler

In [7]:
scaler = StandardScaler()
def standard_scaling(data):
  shape1 = data.shape[1]
  shape2 = data.shape[2]
  data = data.reshape(data.shape[0],-1)
  data = scaler.fit_transform(data)
  data = data.reshape(data.shape[0],shape1,shape2)
  return data

Function for Pytorch dataloaders

In [8]:
def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):
    # convert data dimensions to into to gray scale image format
    if EEGNET: ### EEGNet model requires the last dimension to be 1
        ff = lambda dat: torch.unsqueeze(dat, dim = -1)
    else:
        ff = lambda dat: torch.unsqueeze(dat, dim = 1)

    x_train, x_valid, x_test =  map(ff,(dat['xtrain'],dat['xvalid'],dat['xtest']))
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)

    # TensorDataset & Dataloader
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev))
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev))

    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader},
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})
    return output

In [10]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Pooled
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading pooled data')
dat = load_pooled(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)


xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

print("Shape after data augmentation :", xtrain.shape)
dat['xtrain'], dat['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

print(X_train.shape)
dat = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
           ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')

dat['xtrain']=standard_scaling(dat['xtrain'])
dat['xvalid']=standard_scaling(dat['xvalid'])
dat['xtest']=standard_scaling(dat['xtest'])
dat['xtrain']=torch.tensor(dat['xtrain'])
dat['xvalid']=torch.tensor(dat['xvalid'])
dat['xtest']=torch.tensor(dat['xtest'])
dat['xtrain']=dat['xtrain'].type('torch.FloatTensor')
dat['xvalid']=dat['xvalid'].type('torch.FloatTensor')
dat['xtest']=dat['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')

dat = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading pooled data

Data augmentation
Shape after data augmentation : torch.Size([8866, 3, 321])

Data cropping
torch.Size([26598, 3, 120])

Preprocessing

Dataloaders
Input data shape torch.Size([26598, 1, 3, 120])


# CNN architecture




In [11]:
class CNN(torch.nn.Module):
    def __init__(self, input_size, kernel_size, conv_channels,
                 dense_size, dropout):
        super(CNN, self).__init__()
        self.cconv   = []
        self.MaxPool = nn.MaxPool2d((1, 2), (1, 2))
        self.ReLU    = nn.ReLU()
        self.Dropout = nn.Dropout(dropout)
        self.batchnorm = []
        self.outputs   = 2 # number of output nodes or classess
        # ############ batchnorm ###########
        for jj in conv_channels:
            self.batchnorm.append(nn.BatchNorm2d(jj, eps=0.001, momentum=0.01,
                                                 affine=True, track_running_stats=True).cuda())
        ii = 0 ##### define CONV layer architecture: #####
        for in_channels, out_channels in zip(conv_channels, conv_channels[1:]):
            conv_i = torch.nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                                     kernel_size = kernel_size[ii], #stride = (1, 2),
                                     padding = (kernel_size[ii][0]//2, kernel_size[ii][1]//2))
            self.cconv.append(conv_i)
            self.add_module('CNN_K{}_O{}'.format(kernel_size[ii], out_channels), conv_i)
            ii += 1
        self.flat_dim = self.get_output_dim(input_size, self.cconv)
        self.fc1 = torch.nn.Linear(self.flat_dim, dense_size)
        self.fc2 = torch.nn.Linear(dense_size, self.outputs)

    def get_output_dim(self, input_size, cconv):
        with torch.no_grad():
            input = torch.ones(1,*input_size)
            for conv_i in cconv:
                input = self.MaxPool(conv_i(input))
                flatout = int(np.prod(input.size()[1:]))
                print("Input shape : {} and flattened : {}".format(input.shape, flatout))
        return flatout

    def forward(self, input):
        for jj, conv_i in enumerate(self.cconv):
            input = conv_i(input)
            input = self.batchnorm[jj+1](input)
            input = self.ReLU(input)
            input = self.MaxPool(input)
        # flatten the CNN output
        out = input.view(-1, self.flat_dim)
        out = self.fc1(out)
        out = self.Dropout(out)
        out = self.fc2(out)
        return out

# Training loop

In [12]:
#Hyperparameter settings
num_epochs = 75
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])

description = 'BNCI2014004'

#% get input size (channel x timepoints)
input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

kernel_size = [(3, 8), (3, 8),(3, 8),(3, 8),(3, 8),(3,8)]
conv_chan = [1, 128, 64, 32, 16, 8]

model = CNN(input_size    = input_size,
              kernel_size   = kernel_size,
              conv_channels = conv_chan,
              dense_size    = 256,
              dropout       = 0.5)

# optimizer and the loss function definition
optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
criterion = torch.nn.CrossEntropyLoss()

model.to(dev)
criterion.to(dev)


best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


test_samples = 100
x_test = dat['test_data']['x_test'][:test_samples,:,:]
y_test = dat['test_data']['y_test'][:test_samples]

# predict test data
preds = best_model(x_test.to(dev))
preds_class = preds.data.max(1)[1]

# get the accuracy
corrects = torch.sum(preds_class == y_test.data.to(dev))
test_acc = corrects.cpu().numpy()/x_test.shape[0]
print("Test Accuracy :", test_acc)

# save results
tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

table.loc[description] = tab
val_acc = np.max(val_accs)
print('\n---------------------------------------------------------------------\n')

Input shape : torch.Size([1, 128, 3, 60]) and flattened : 23040
Input shape : torch.Size([1, 64, 3, 30]) and flattened : 5760
Input shape : torch.Size([1, 32, 3, 15]) and flattened : 1440
Input shape : torch.Size([1, 16, 3, 8]) and flattened : 384
Input shape : torch.Size([1, 8, 3, 4]) and flattened : 96
Epoch 1/75
train loss: 0.0108, acc: 0.5393
val loss: 0.0096, acc: 0.6450
Epoch 2/75
train loss: 0.0101, acc: 0.5979
val loss: 0.0094, acc: 0.6625
Epoch 3/75
train loss: 0.0099, acc: 0.6151
val loss: 0.0094, acc: 0.6598
Epoch 4/75
train loss: 0.0098, acc: 0.6261
val loss: 0.0095, acc: 0.6643
Epoch 5/75
train loss: 0.0096, acc: 0.6488
val loss: 0.0095, acc: 0.6613
Epoch 6/75
train loss: 0.0093, acc: 0.6713
val loss: 0.0099, acc: 0.6543
Epoch 7/75
train loss: 0.0087, acc: 0.7069
val loss: 0.0103, acc: 0.6450
Epoch 8/75
train loss: 0.0077, acc: 0.7498
val loss: 0.0118, acc: 0.6592
Epoch 9/75
train loss: 0.0066, acc: 0.7875
val loss: 0.0121, acc: 0.6567
Epoch 10/75
train loss: 0.0057, acc: 

# Save and print results

In [13]:
table.to_csv('results/BNCI2014004_cnn_pooled.csv')

print(table)

             Train_Acc   Val_Acc  Test_Acc  Epoch
BNCI2014004   0.626137  0.664262      0.67      4


Repeat the same steps for other datasets
## 2. BNCI2014001

In [14]:
itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


In [15]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Pooled
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading pooled data')
dat = load_pooled(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.000001,  multiple = 2)


xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

print("Shape after data augmentation :", xtrain.shape)
dat['xtrain'], dat['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

print(X_train.shape)
dat = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
           ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')

dat['xtrain']=standard_scaling(dat['xtrain'])
dat['xvalid']=standard_scaling(dat['xvalid'])
dat['xtest']=standard_scaling(dat['xtest'])
dat['xtrain']=torch.tensor(dat['xtrain'])
dat['xvalid']=torch.tensor(dat['xvalid'])
dat['xtest']=torch.tensor(dat['xtest'])
dat['xtrain']=dat['xtrain'].type('torch.FloatTensor')
dat['xvalid']=dat['xvalid'].type('torch.FloatTensor')
dat['xtest']=dat['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')

dat = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading pooled data

Data augmentation
Shape after data augmentation : torch.Size([3524, 22, 321])

Data cropping
torch.Size([10572, 22, 120])

Preprocessing

Dataloaders
Input data shape torch.Size([10572, 1, 22, 120])


# ResNet50 architecture

ResNet uses residual blocks

In [16]:
class ResBlock(nn.Module):

    def __init__(self, in_channels, inter_channels, identity_downsample=None, stride=1):
        super(ResBlock, self).__init__()
        self.expansion=4
        self.in_channels = in_channels
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=inter_channels,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(inter_channels)
        self.conv2 = nn.Conv2d(
            in_channels=inter_channels,
            out_channels=inter_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(inter_channels)
        self.conv3 = nn.Conv2d(
            in_channels=inter_channels,
            out_channels=inter_channels*self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False,
        )
        self.bn3 = nn.BatchNorm2d(inter_channels * self.expansion)
        self.relu = nn.LeakyReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity=x.clone()
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        out += identity
        out = self.relu(out)
        return out



class ResNet50(nn.Module):

    def __init__(self, classes=None):
        super(ResNet50, self).__init__()
        self.in_channels = 64
        self.conv1=nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.r=nn.LeakyReLU()
        self.mp=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.l1=self._make_layer(ResBlock, 3, inter_channels=64, stride=1)
        self.l2=self._make_layer(ResBlock, 4, inter_channels=128, stride=2)
        self.l3=self._make_layer(ResBlock, 6, inter_channels=256, stride=2)
        self.l4=self._make_layer(ResBlock, 3, inter_channels=512, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * 4, 100)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(100, classes)

    def forward(self, x):
      x = self.conv1(x)
      x = self.bn1(x)
      x = self.r(x)
      x = self.mp(x)
      x = self.l1(x)
      x = self.l2(x)
      x = self.l3(x)
      x = self.l4(x)
      x = self.avgpool(x)
      x = x.reshape(x.shape[0], -1)
      x = self.fc1(x)
      x=self.Dropout(x)
      x = self.fc2(x)

      return x

    def _make_layer(self, block, num_residual_blocks, inter_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != inter_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    inter_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(inter_channels * 4),
            )

        layers.append(
            block(self.in_channels, inter_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50
        self.in_channels = inter_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, inter_channels))

        return nn.Sequential(*layers)

In [17]:
#Hyperparameter settings
num_epochs = 50
learning_rate = 1e-3
weight_decay = 1e-3
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])

description = 'BNCI2014001'

#% get input size (channel x timepoints)
input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])


model = ResNet50(classes=2)

# optimizer and the loss function definition
optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
criterion = torch.nn.CrossEntropyLoss()

model.to(dev)
criterion.to(dev)


best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


test_samples = 100
x_test = dat['test_data']['x_test'][:test_samples,:,:]
y_test = dat['test_data']['y_test'][:test_samples]

# predict test data
preds = best_model(x_test.to(dev))
preds_class = preds.data.max(1)[1]

# get the accuracy
corrects = torch.sum(preds_class == y_test.data.to(dev))
test_acc = corrects.cpu().numpy()/x_test.shape[0]
print("Test Accuracy :", test_acc)

# save results
tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

table.loc[description] = tab
val_acc = np.max(val_accs)
print('\n---------------------------------------------------------------------\n')

Epoch 1/50
train loss: 0.0139, acc: 0.4963
val loss: 0.0116, acc: 0.4989
Epoch 2/50
train loss: 0.0112, acc: 0.5105
val loss: 0.0110, acc: 0.4989
Epoch 3/50
train loss: 0.0110, acc: 0.5101
val loss: 0.0111, acc: 0.4989
Epoch 4/50
train loss: 0.0110, acc: 0.5044
val loss: 0.0110, acc: 0.5117
Epoch 5/50
train loss: 0.0109, acc: 0.5072
val loss: 0.0109, acc: 0.5540
Epoch 6/50
train loss: 0.0109, acc: 0.5161
val loss: 0.0124, acc: 0.5480
Epoch 7/50
train loss: 0.0103, acc: 0.6011
val loss: 0.0104, acc: 0.6304
Epoch 8/50
train loss: 0.0096, acc: 0.6557
val loss: 0.0111, acc: 0.5843
Epoch 9/50
train loss: 0.0091, acc: 0.6792
val loss: 0.0100, acc: 0.6251
Epoch 10/50
train loss: 0.0088, acc: 0.7094
val loss: 0.0099, acc: 0.6470
Epoch 11/50
train loss: 0.0085, acc: 0.7212
val loss: 0.0101, acc: 0.6447
Epoch 12/50
train loss: 0.0081, acc: 0.7375
val loss: 0.0101, acc: 0.6168
Epoch 13/50
train loss: 0.0078, acc: 0.7545
val loss: 0.0124, acc: 0.6100
Epoch 14/50
train loss: 0.0073, acc: 0.7760
val

In [18]:
table.to_csv('BNCI2014001_resnet50_pooled.csv')

print(table)

             Train_Acc   Val_Acc  Test_Acc  Epoch
BNCI2014001   0.884412  0.656085      0.69     21


## 3. Weibo2014

In [19]:
itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


In [20]:
#Filter
print('\nFiltering from 8 to 15 Hz')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 15)

#--------------------------------------------------------------------------
#Pooled
subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nLoading pooled data')
dat = load_pooled(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data
print('\nData augmentation')
augmdata = dict(std_dev = 0.00001,  multiple = 2)


xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'],
                                     std_dev = augmdata['std_dev'],
                                     multiple = augmdata['multiple'])

print("Shape after data augmentation :", xtrain.shape)
dat['xtrain'], dat['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nData cropping')
fs = 80 # sampling frequency
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'],
                                    dat['ytrain'],xpercent = 50)
X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'],
                                    dat['yvalid'],xpercent = 50)
X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'],
                                    dat['ytest'], xpercent = 50)

print(X_train.shape)
dat = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
           ytrain = y_train, yvalid = y_valid, ytest = y_test)

#--------------------------------------------------------------------------
# Standard scaling
print('\nPreprocessing')

dat['xtrain']=standard_scaling(dat['xtrain'])
dat['xvalid']=standard_scaling(dat['xvalid'])
dat['xtest']=standard_scaling(dat['xtest'])
dat['xtrain']=torch.tensor(dat['xtrain'])
dat['xvalid']=torch.tensor(dat['xvalid'])
dat['xtest']=torch.tensor(dat['xtest'])
dat['xtrain']=dat['xtrain'].type('torch.FloatTensor')
dat['xvalid']=dat['xvalid'].type('torch.FloatTensor')
dat['xtest']=dat['xtest'].type('torch.FloatTensor')

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')

dat = get_data_loaders(dat, batch_size = 64)


Filtering from 8 to 15 Hz


/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.10/dist-packages/mne/filter.py:312: Dep


Loading pooled data

Data augmentation
Shape after data augmentation : torch.Size([2148, 60, 321])

Data cropping
torch.Size([6444, 60, 120])

Preprocessing

Dataloaders
Input data shape torch.Size([6444, 1, 60, 120])


In [21]:
#Hyperparameter settings
num_epochs = 75
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 64
verbose = 2


#% used to save the results table
results = {}
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch'])

description = 'Weibo2014'

#% get input size (channel x timepoints)
input_size = (1, dat['test_data']['x_test'].shape[-2],
                 dat['test_data']['x_test'].shape[-1])

kernel_size = [(3, 8), (3, 8),(3, 8),(3, 8),(3, 8),(3,8)]
conv_chan = [1, 128, 64, 32, 16, 8]

model = CNN(input_size    = input_size,
              kernel_size   = kernel_size,
              conv_channels = conv_chan,
              dense_size    = 256,
              dropout       = 0.5)

# optimizer and the loss function definition
optimizer = torch.optim.Adam(model.parameters(),
                             lr = learning_rate,
                             weight_decay = weight_decay)
criterion = torch.nn.CrossEntropyLoss()

model.to(dev)
criterion.to(dev)


best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'],
                  criterion, optimizer, dev=dev, lr_scheduler=False,
                  num_epochs = num_epochs, verbose = verbose)


test_samples = 100
x_test = dat['test_data']['x_test'][:test_samples,:,:]
y_test = dat['test_data']['y_test'][:test_samples]

# predict test data
preds = best_model(x_test.to(dev))
preds_class = preds.data.max(1)[1]

# get the accuracy
corrects = torch.sum(preds_class == y_test.data.to(dev))
test_acc = corrects.cpu().numpy()/x_test.shape[0]
print("Test Accuracy :", test_acc)

# save results
tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)

table.loc[description] = tab
val_acc = np.max(val_accs)
print('\n---------------------------------------------------------------------\n')

Input shape : torch.Size([1, 128, 60, 60]) and flattened : 460800
Input shape : torch.Size([1, 64, 60, 30]) and flattened : 115200
Input shape : torch.Size([1, 32, 60, 15]) and flattened : 28800
Input shape : torch.Size([1, 16, 60, 8]) and flattened : 7680
Input shape : torch.Size([1, 8, 60, 4]) and flattened : 1920
Epoch 1/75
train loss: 0.0112, acc: 0.5300
val loss: 0.0107, acc: 0.6047
Epoch 2/75
train loss: 0.0103, acc: 0.6130
val loss: 0.0103, acc: 0.6208
Epoch 3/75
train loss: 0.0097, acc: 0.6560
val loss: 0.0105, acc: 0.6047
Epoch 4/75
train loss: 0.0088, acc: 0.7058
val loss: 0.0115, acc: 0.5849
Epoch 5/75
train loss: 0.0075, acc: 0.7756
val loss: 0.0115, acc: 0.6022
Epoch 6/75
train loss: 0.0061, acc: 0.8237
val loss: 0.0123, acc: 0.5973
Epoch 7/75
train loss: 0.0050, acc: 0.8625
val loss: 0.0141, acc: 0.5725
Epoch 8/75
train loss: 0.0040, acc: 0.8973
val loss: 0.0150, acc: 0.5874
Epoch 9/75
train loss: 0.0033, acc: 0.9178
val loss: 0.0165, acc: 0.5750
Epoch 10/75
train loss: 0

In [22]:
table.to_csv('Weibo2014_cnn_pooled.csv')

print(table)

           Train_Acc   Val_Acc  Test_Acc  Epoch
Weibo2014   0.612973  0.620818      0.62      2
